In [1]:
import sys
sys.path.append('../src/')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import json
import scipy
from antenna import Antenna
import itertools
from tqdm import tqdm

In [2]:
with open("../config.json", 'r') as cfg:
    params = json.load(cfg)

antenna_params = params["antenna_params"]
opt_params = params["optimisation_params"]

In [3]:
antenna_params['N'] = 12
antenna_params["wavelengths"]=[1, 2]

In [4]:
antenna = Antenna(antenna_params)
antenna.set_objective(weights=[1.0, 1.0])
antenna.set_jacobian(weights=[1.0, 1.0])
antenna.set_hessian(weights=[1.0, 1.0])

In [50]:
B = np.block([[0*np.eye(2*(antenna.N - 1)), np.eye(2*(antenna.N - 1))], 
              [np.eye(2*(antenna.N - 1))[::-1], 0*np.eye(2*(antenna.N - 1))]])

In [77]:
def A_matrix(N, k, d, phi, phi0):
    M = phi.shape[0]
    temp = []
    for i in range(M):
        temp_j = []
        for j in range(N - 1):
            temp_ij = np.exp(1j * k * j * d * np.cos(phi[i])) - np.exp(1j * k * (N - 1) * d * (np.cos(phi[i]) - np.cos(phi0)))*np.exp(1j * k * j * d * np.cos(phi0)) 
            temp_j.append(np.array([[np.real(temp_ij), -np.imag(temp_ij)], [np.imag(temp_ij), np.real(temp_ij)]]))
        temp.append(temp_j)
    A = np.block(temp)
    return A

def b_vector(N, k, d, phi, phi0):
    M = phi.shape[0]
    temp = []
    for i in range(M):
        temp_i = np.exp(1j * k * (N - 1) * d * (np.cos(phi[i]) - np.cos(phi0)))
        temp.append([np.real(temp_i), np.imag(temp_i)])
    b = np.block(temp).reshape(-1, 1)
    return b

In [78]:
A1 = A_matrix(antenna.N, 2*np.pi/antenna.lambdas[0], 
                           antenna.d, 
                           np.radians(np.arange(0, 180 + antenna.beam_resolution, antenna.beam_resolution)), 
                           np.pi/2)

b1 = b_vector(antenna.N, 2*np.pi/antenna.lambdas[0], 
             antenna.d, 
             np.radians(np.arange(0, 180 + antenna.beam_resolution, antenna.beam_resolution)), 
             np.pi/2)

A2 = A_matrix(antenna.N, 2*np.pi/antenna.lambdas[1], 
                           antenna.d, 
                           np.radians(np.arange(0, 180 + antenna.beam_resolution, antenna.beam_resolution)), 
                           np.pi/2)

b2 = b_vector(antenna.N, 2*np.pi/antenna.lambdas[1], 
             antenna.d, 
             np.radians(np.arange(0, 180 + antenna.beam_resolution, antenna.beam_resolution)), 
             np.pi/2)

In [80]:
A = scipy.linalg.block_diag(*[A1, A2])
b = np.concatenate((b1, b2))

In [83]:
A.shape

(724, 44)

In [52]:
def f(x):
    return x.T@B@x

In [103]:
def cons(x):
    return 10 - np.linalg.norm(A@x + b, np.inf)

In [104]:
cons1 = {'type':'ineq', 'fun': cons}
cons2 = {'type':'ineq', 'fun': lambda x: x}

In [105]:
res = scipy.optimize.minimize(f, 
                              x0=np.ones((antenna.N - 1) * antenna.n_currents * 2), 
                              constraints=[cons1, cons2])

In [107]:
res.x

array([-1.96637300e-14, -3.00553607e-14,  6.45542537e-14,  1.46272499e-14,
        3.55257060e-14,  4.56551212e-15,  5.46022771e-14,  1.97578736e-14,
        3.22736886e-14,  2.08081305e-15,  2.98420315e-14,  2.18361403e-14,
        3.55355322e-14,  2.77360998e-14,  2.11893078e-14,  3.63554290e-14,
        4.95938819e-14,  1.44999724e-14,  2.00928070e-14,  2.16824414e-14,
        1.98390766e-14, -2.46252956e-16,  3.86115913e-14,  4.66220806e-16,
       -1.32206626e-14,  8.81824784e-15,  3.59366879e-14,  3.63658661e-14,
        2.30203722e-14,  2.91935420e-15,  1.97317304e-14,  4.44136479e-14,
        5.92669377e-14,  4.17047775e-14,  2.61764611e-14,  3.77867555e-14,
        2.63328005e-14,  2.91235070e-14,  3.75586343e-14,  6.15901668e-14,
        1.74325914e-14,  3.06538860e-14,  1.80083558e-14,  3.02145592e-14])